# Summary
This script fetches the last 100 proposals and puts them in an Excel.

# Instructions
- to make this file yours, click **File -> Save a copy in Drive** (you only need to do this once), then work with the copied file
- set the `network` in the first code block (lower caps, only `polkadot` or `kusama`)
- in the menu above click **Runtime -> Run all**
  - an info box pops up: **Run anyway**
- wait half a minute (the wheels in the code boxes will stop turning when done)
- open the file browser on the left
  - download `{network}.xlsx` (if you don't see the file, click the refresh icon in the file browser)

# Notes
- USD prices of executed proposals are calculated to the exchange rate of the day of the last status change.
- Not every referendum gets a DOT value assigned from Polkassembly. E.g. Bounties are not counted, since the money is not spent. We also see proposals without value where we don't have an explanation yet, e.g. 465

In [ ]:
network = "polkadot"
# network = "kusama"
explorer = "polkassembly"
# explorer = "subsquare"

if network == "polkadot":
  denomination_factor = 1e10
  ticker = "DOT-USD"
else:
  denomination_factor = 1e12
  ticker = "KSM-USD"

# Preconditions

In [ ]:
import requests
import pandas as pd
import json
import datetime
import time

In [ ]:
pip install pandas_datareader --upgrade

# Fetch Data

## Prices

In [ ]:
# Import the yfinance. If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf


def get_historic_dotusd_price():
  # 1. Get today's date
  today = datetime.datetime.now().strftime("%Y-%m-%d")

  # Get the data for the stock AAPL
  data = yf.download(ticker,'2020-08-20',today)
  return data

dotusd_historic_df = get_historic_dotusd_price()
dotusd_historic_df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-20,12.221921,14.720971,11.918807,14.713737,14.713737,10219917
2020-08-21,14.707350,15.748715,13.295981,14.951648,14.951648,14174411
2020-08-22,14.950848,19.069290,14.409958,18.392479,18.392479,21241566
2020-08-23,18.396067,18.574581,16.690157,17.653250,17.653250,11197391
2020-08-24,17.666306,23.100573,17.296612,22.919081,22.919081,24548411
...,...,...,...,...,...,...
2024-02-22,45.777130,47.505398,44.697109,45.732670,45.732670,19133894
2024-02-23,45.729897,46.428673,44.351349,45.315746,45.315746,22398050
2024-02-24,45.315746,47.426479,44.264713,47.206139,47.206139,17561243


In [ ]:
def get_current_dotusd_price():
    url = 'https://api.coingecko.com/api/v3/simple/price?ids=polkadot&vs_currencies=usd'
    response = requests.get(url)
    data = response.json()
    return data['polkadot']['usd']

# Get the DOTUSD exchange rate
dotusd_price = get_current_dotusd_price()
print(f"DOT/USD exchange rate: {dotusd_price}")

DOT/USD exchange rate: 8.14


## Fetch Proposals from Polkassembly

In [ ]:


# Define the URL
page_size = 100
track_status = "All"
url = f"https://api.polkassembly.io/api/v1/listing/on-chain-posts?page=1&proposalType=referendums_v2&listingLimit={page_size}&trackStatus={track_status}&sortBy=newest"
headers = {"x-network":network}

# Send a GET request to the URL
response = requests.get(url, headers=headers)


# Check if the request was successful
if response.status_code != 200:
    summary = "Failed to retrieve data. Status code: {}".format(response.status_code)
    print(summary)
    exit()




In [ ]:

data = response.json()
# normalize into df
raw = pd.DataFrame(data)
raw_df = pd.json_normalize(raw['posts'])
raw

,count,posts
0,352,"{'beneficiaries': [], 'comments_count': 0, 'cr..."
1,352,"{'beneficiaries': [], 'comments_count': 0, 'cr..."
2,352,"{'beneficiaries': [], 'comments_count': 1, 'cr..."
3,352,"{'beneficiaries': [], 'comments_count': 1, 'cr..."
4,352,"{'beneficiaries': [], 'comments_count': 0, 'cr..."
...,...,...
95,352,{'beneficiaries': [{'value': 'DuLr6CeLXezrfumF...
96,352,{'beneficiaries': [{'value': 'Et9M3rrA7H2kHQEG...
97,352,"{'beneficiaries': [], 'comments_count': 0, 'cr..."
98,352,"{'beneficiaries': [], 'comments_count': 2, 'cr..."


In [ ]:
raw_df.columns

Index(['beneficiaries', 'comments_count', 'created_at', 'curator',
       'description', 'end', 'hash', 'identity', 'isSpam',
       'isSpamReportInvalid', 'method', 'parent_bounty_index', 'post_id',
       'proposalHashBlock', 'proposer', 'spam_users_count', 'status',
       'status_history', 'tags', 'timeline', 'title', 'track_no', 'type',
       'user_id', 'post_reactions.👍', 'post_reactions.👎', 'tally.ayes',
       'tally.nays', 'tally.support', 'topic.id', 'topic.name',
       'requestedAmount'],
      dtype='object')

In [ ]:
raw_df["status"].unique()

array(['Deciding', 'Executed', 'Rejected', 'TimedOut', 'Confirmed',
       'ExecutionFailed'], dtype=object)

# Transform


In [ ]:
# | index | Title | Status | USD | DOT | Comment | Phase | End Time | Propose Time | Beneficiary | Category | Subcategory | Subsquare | Polkassembly | Subscan |


df = raw_df.copy()

from datetime import datetime

# Define your ID to Origin mapping
id_to_origin_mapping = {
    0: 'Root',
    1: 'Whitelisted Caller',
    10: 'Staking Admin',
    11: 'Treasurer',
    12: 'Lease Admin',
    13: 'Fellowship Admin',
    14: 'General Admin',
    15: 'Auction Admin',
    20: 'Referendum Canceller',
    21: 'Referendum Killer',
    30: 'Small Tipper',
    31: 'Big Tipper',
    32: 'Small Spender',
    33: 'Medium Spender',
    34: 'Big Spender'
}

def determine_usd_price(row):
  statuses_where_i_want_to_get_the_historic_price = ["Executed"]
  if row["status"] in statuses_where_i_want_to_get_the_historic_price:
    executed_date = row["last_status_change"] # pd.to_datetime(
    # Find the closest matching date in dotusd_historic_df
    closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')
    conversion_rate = dotusd_historic_df.iloc[closest_date]['Close']
    return row["DOT"] * conversion_rate
  else:
    return row["DOT"] * dotusd_price

# Function to format USD amounts
def format_currency(amount):
    if amount >= 1_000_000:
        return '{:.1f}m'.format(amount / 1_000_000)
    elif amount >= 1_000:
        return '{:.0f}k'.format(amount / 1_000)
    else:
        return '{:.0f}'.format(amount)

# Function to format date to quarter and year
def format_date_to_quarter(date):
    if pd.isnull(date):
        return None
    quarter = (date.month - 1) // 3 + 1
    return f"Q{quarter}-{date.year}"

# Create a copy of the raw dataframe to work with
df = raw_df.copy()

# Build columns
df["last_status_change"] = pd.to_datetime(df["status_history"].apply(lambda x: x[-1]["timestamp"] if len(x) > 0 else None))
df["last_status_change_formatted"] = df["last_status_change"].apply(format_date_to_quarter)
df["DOT"] = (pd.to_numeric(df["requestedAmount"]) / denomination_factor)
df["DOT_formatted"] = df["DOT"].apply(format_currency)
df["USD"] = df.apply(determine_usd_price, axis=1)
df["USD_formatted"] = df["USD"].apply(format_currency)
df["Status"] = df["status"]
df["index"] = df["post_id"].apply(lambda x:f'=HYPERLINK("https://{network}.{explorer}.io/referenda/{x}", {x})')


# Replace the 'Track' column with the mapping from IDs to Origin names
df["Track"] = df["track_no"].map(id_to_origin_mapping)

# Format the 'last_status_change' column to 'Q{quarter}-{year}'
df["last_status_change_formatted"] = df["last_status_change"].apply(format_date_to_quarter)


# More filtering
df = df.set_index("post_id")
df = df[["index", "title", "DOT_formatted", "USD_formatted", "Status", "Track", "last_status_change_formatted", "DOT", "USD"]]

#df[df["Status"] == "Executed"]
df


<ipython-input-136-123d4edec94b>:32: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')
<ipython-input-136-123d4edec94b>:32: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')


,index,title,DOT_formatted,USD_formatted,Status,Track,last_status_change_formatted,DOT,USD
post_id,,,,,,,,,
351,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",HRMP channel between Encointer and Asset Hub,nan,nan,Deciding,General Admin,Q1-2024,NaN,NaN
350,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",Bounty Closure Proposal: n.16 and n.21,nan,nan,Deciding,Treasurer,Q1-2024,NaN,NaN
349,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",Virto 2024,nan,nan,Deciding,Treasurer,Q1-2024,NaN,NaN
348,"=HYPERLINK(""https://polkadot.subsquare.io/refe...","Set `safeXcmVersion` to 3 for AssetHub, Bridge...",nan,nan,Deciding,Whitelisted Caller,Q1-2024,NaN,NaN
347,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",Open HRMP channel between Kreivo and AssetHub,nan,nan,Deciding,General Admin,Q1-2024,NaN,NaN
...,...,...,...,...,...,...,...,...,...
256,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",Kusama RPC services Dwellir Q3 2023,810,16k,Executed,Medium Spender,Q3-2023,809.700,15734.288983
255,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",test,665,5k,TimedOut,Medium Spender,Q3-2023,665.000,5413.100000
254,"=HYPERLINK(""https://polkadot.subsquare.io/refe...",SmallTipper,nan,nan,Executed,Small Tipper,Q3-2023,NaN,NaN


# Export
## Export to Excel

In [ ]:
!pip install openpyxl


In [ ]:
# Specify the filename
filename = f'{network}.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(filename, engine='openpyxl', index=False)

print(f'DataFrame has been exported to {filename}')


DataFrame has been exported to kusama.xlsx


# Ignore

In [ ]:
output = '''
# Connect to Google Drive
from google.colab import auth
import gspread
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
# Read the Treasury Spreadsheet
import pandas as pd
worksheet = gc.open('Scrape').sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame(rows[1:], columns=rows[0])
df
'''

raw_df[raw_df["requestedAmount"].isna()][["method", "post_id"]]

,method,post_id
0,batch_all,351
1,batch_all,350
2,NaN,349
3,dispatch_whitelisted_call_with_preimage,348
4,NaN,347
...,...,...
91,dispatch_whitelisted_call_with_preimage,260
93,dispatch_whitelisted_call_with_preimage,258
94,dispatch_whitelisted_call_with_preimage,257
97,NaN,254
